## Testing  SAM
- speed when using differnt prediction methods
- different sam versions

In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

/home/pvondrlik/Anwendungen/miniconda3/envs/ba-cv-sam-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-12 10:21:21.445955: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-12 10:21:21.499546: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-12 10:21:21.755289: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 10:21:21.755408: E tensorflow/compiler/xla/stream_execu

In [3]:
def show_mask(mask_data, ax):
    color = np.array([1, 0, 1, 0.6])
    h, w = mask_data.shape[-2:]
    mask_image = mask_data.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(pos_points, ax, marker_size=375):
    ax.scatter(pos_points[0], pos_points[1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def display_sam(mask, point, path):
    
    image = cv2.imread(path)
    fig, ax = plt.subplots(figsize=(5,3)) # Adjust size as needed
    # Display the image
    ax.imshow(image)
    # Assuming show_mask and show_points are functions that overlay masks and points respectively

    show_mask(mask, ax)  # Show the first mask
    show_points(point, ax)

    plt.title('Expl_1_ET_1_2023-09-05_11-56-16_ET')
    plt.show()

In [4]:
from ultralytics import SAM

#sam model 
model = SAM('/home/pvondrlik/Desktop/BA_Thesis/repo-movie-analysis/models/mobile_sam.pt')
model.info()
sam = model

Model summary: 346 layers, 10130092 parameters, 10130092 gradients


In [5]:
from ultralytics.models.sam import Predictor as SAMPredictor
overrides = dict(conf=0.25, task='segment', mode='predict', imgsz=1024, model="/home/pvondrlik/Desktop/BA_Thesis/repo-movie-analysis/models/mobile_sam.pt")
predictor = SAMPredictor(overrides=overrides)

### testing sam predicitons for speed

In [25]:
from PIL import Image
# resize image to 256x256
path = "/home/pvondrlik/Desktop/BA_Thesis/repo-movie-analysis/data/Expl_1_ET_1_2023-09-05_11-56-16_ET/frame_11110.jpg"
path_new = "/home/pvondrlik/Desktop/BA_Thesis/repo-movie-analysis/data/Expl_1_ET_1_2023-09-05_11-56-16_ET/frame_11110_small.jpg"
img = Image.open(path)
img = img.resize((256, 256))
img.save(path_new)

In [10]:
import time
path = "/home/pvondrlik/Desktop/BA_Thesis/repo-movie-analysis/data/Expl_3_ET_1_2023-09-06_13-24-43_ET/video_frames_img/frame_13379.jpg"
points = [[100, 100], [200, 200],[100, 100], [200, 200], [100, 100], [200, 200],[100, 100] ]
labels=[1, 1, 1, 1, 1, 1, 1 ]

In [ ]:
# # more efficient but still slow
starttime = time.process_time()
gen_res = sam(source = path, stream=False, points=points, labels=labels)
print(f'Time sam 7 points: {time.process_time() - starttime}')
#r[0].masks.data[0]

# inefficient
starttime = time.process_time()
for p in points:
    res_sam = sam(path, points=p, labels=[1])
print(f'Time 7 times sam : {time.process_time() - starttime}')

# predictor
r1 = []
starttime = time.process_time()
predictor.set_image(path)
for p in points:
    r1.append(predictor(im, points=points, labels=labels, return_tensors=True))
predictor.reset_image()
print(f'Time predictor 7 points: {time.process_time() - starttime}')


### show the video

In [ ]:
video_path = "video_raw.mp4"
cap = cv2.VideoCapture(video_path)

# get video properties
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

# the cap.set is not working. therfore I use this one
for frame in range(1000):  #TODO: fangen frames by 0 ode 1 an
    ret, frame = cap.read()
    if not ret:
        break


# Loop through frames
for frame_nr in range(1000, 1010):

    ret, frame = cap.read() # 
    results = sam(source = frame, stream=False, points=points, labels=labels)
    
    for i in range(len(results[0].masks.data)):
        mask = results[0].masks.data[i]
        print(mask)
        ret, frame = cap.read()

### second sam approach


In [17]:
import torch
from PIL import Image
import numpy as np
import requests
from transformers import SamModel, SamProcessor
from ultralytics import SAM
import cv2
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor


Model summary: 238 layers, 93735472 parameters, 93735472 gradients


In [16]:
model = SAM('/models/sam_b.pt')
model.info()
sam = model
img_path =  ""
img = cv2.imread(img_path)
input_point = np.array([200, 200])
input_label = np.array(1)
results = model(img_path, points=input_point, labels=input_label, return_tensors=True)
masks1, scores1 = results[0].masks, results[0].probs
masks1, scores1